### Ingestion file Movie Genre

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment =  dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date =  dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

In [0]:
movie_genre_schema = StructType([
    StructField("movieId", IntegerType(), False),
    StructField("genreId", IntegerType(), False)
])

In [0]:
movie_genre = spark.read.schema(movie_genre_schema).json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")


In [0]:
movie_genre_df = movie_genre\
    .withColumnRenamed("movieId", "movie_id")\
    .withColumnRenamed("genreId", "genre_id")\
    .withColumn("ingestion_date", current_timestamp())\
    .withColumn("environment", lit("PROD"))\
    .withColumn("file_date", lit(v_file_date))

In [0]:
# movie_genre_df.write.mode("append").partitionBy("file_date").format("delta").saveAsTable("movie_silver.movie_genres")
# overwrite_partition(movie_genre_df, "movie_silver", "movie_genres", "file_date")

merge_condition = "tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date"
merge_delta_lake(movie_genre_df, "movie_silver", "movie_genres", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
select file_date, count(1) 
from movie_silver.movie_genres
group by file_date;

In [0]:
%sql
select *
from movie_silver.movie_genres;

In [0]:
dbutils.notebook.exit("Success")